Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [ ]:
!git clone https://github.com/neonbjb/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!python3 setup.py install

In [1]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HF hub.
tts = TextToSpeech()

/Users/tanishsurana/opt/anaconda3/envs/tortoise/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 2.06k/2.06k [00:00<00:00, 3.07MB/s]
Downloading: 100%|██████████| 1.18G/1.18G [00:32<00:00, 39.3MB/s]
Downloading: 100%|██████████| 159/159 [00:00<00:00, 208kB/s]
Downloading: 100%|██████████| 1.57k/1.57k [00:00<00:00, 7.19MB/s]
Downloading: 100%|██████████| 181/181 [00:00<00:00, 1.37MB/s]
Downloading: 100%|██████████| 85.0/85.0 [00:00<00:00, 380kB/s]


100% |########################################################################|


Done.


100% |########################################################################|


Done.


100% |########################################################################|


Done.


100% |########################################################################|


Done.


In [8]:
# This is the text that will be spoken.
text = "Hi, this is a test and I don't know how well this will work"

# Here's something for the poetically inclined.. (set text=)
"""
Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "fast"

In [16]:
# Tortoise will attempt to mimic voices you provide. It comes pre-packaged
# with some voices you might recognize.

# Let's list all the voices available. These are just some random clips I've gathered
# from the internet as well as a few voices from the training dataset.
# Feel free to add your own clips to the voices/ folder.
%ls tortoise/voices

IPython.display.Audio('tortoise/voices/freeman/1.wav')

angie/               lj/                  train_daws/
applejack/           mol/                 train_dotrice/
cond_latent_example/ myself/              train_dreams/
daniel/              pat/                 train_empire/
deniro/              pat2/                train_grace/
emma/                rainbow/             train_kennard/
freeman/             snakes/              train_lescault/
geralt/              tim_reynolds/        train_mouse/
halle/               tom/                 weaver/
jlaw/                train_atkins/        william/


In [4]:
# Pick one of the voices from the output above
voice = 'tom'

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

Generating autoregressive samples..


100%|██████████| 96/96 [23:13<00:00, 14.52s/it]


Computing best candidates using CLVP


100%|██████████| 96/96 [02:23<00:00,  1.50s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [05:26<00:00,  4.08s/it]


In [19]:
# Optionally, upload use your own voice by running the next two cells. I recommend
# you upload at least 2 audio clips. They must be a WAV file, 6-10 seconds long.
CUSTOM_VOICE_NAME = "daily"
text = "What kind of hawk is this? It's a python, foreigner"

# Generate speech with the custotm voice.
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

Generating autoregressive samples..


100%|██████████| 96/96 [17:40<00:00, 11.05s/it]


Computing best candidates using CLVP


100%|██████████| 96/96 [03:31<00:00,  2.20s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [03:02<00:00,  2.28s/it]
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


In [5]:
# Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
# (Note: random voices can be prone to strange utterances)
gen = tts.tts_with_preset(text, voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

100% |########################################################################|


Done.


100% |########################################################################|


Done.
Generating autoregressive samples..


  5%|▌         | 5/96 [01:05<19:50, 13.08s/it]


KeyboardInterrupt: 

In [ ]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(['pat', 'william'])

gen = tts.tts_with_preset("They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.", 
                          voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('captain_kirkard.wav')

In [ ]:
del tts  # Will break other cells, but necessary to conserve RAM if you want to run this cell.

# Tortoise comes with some scripts that does a lot of the lifting for you. For example,
# read.py will read a text file for you.
!python3 tortoise/read.py --voice=train_atkins --textfile=tortoise/data/riding_hood.txt --preset=ultra_fast --output_path=.

IPython.display.Audio('train_atkins/combined.wav')
# This will take awhile..